In [2]:
import IPython.display as ipd
import librosa
import librosa.display
import librosa.feature
import matplotlib.pyplot as plt
from PIL import Image

from scipy import signal

from tsfresh import extract_features, extract_relevant_features, select_features
from tsfresh.utilities.dataframe_functions import impute
from tsfresh.feature_extraction.settings import from_columns
from tsfresh.feature_extraction import ComprehensiveFCParameters, EfficientFCParameters, MinimalFCParameters

import json
from pathlib import Path

import numpy as np
import pandas as pd

from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn_pandas import DataFrameMapper

from tqdm import tqdm

from concurrent.futures import ProcessPoolExecutor

import pickle

ROOT_PATH = Path("..")

## Get ids, labels, and train-test splits

In [3]:
df = pd.read_csv(ROOT_PATH / "data/raw/metadata.csv")
# svc - song vs call ids
# filter ids -> <20s, quality A & B
# svc ids -> only rows that have call or song (not both)
filter_ids = pd.read_json(ROOT_PATH / "data/raw/filter_ids.json").squeeze()
svc_ids = pd.read_json(ROOT_PATH / "data/raw/song_vs_call.json").squeeze()
svc_df = df.loc[df.id.isin(svc_ids)].copy()
# set index to id
svc_df.set_index('id', inplace=True)

with open(ROOT_PATH / "data/processed/svc_split.json") as svc_split_file:
    svc_split = json.load(svc_split_file)
    train_ids = svc_split["train_ids"]
    test_ids = svc_split["test_ids"]

In [4]:
# Add response variable
type_col = svc_df.type.str.lower().str.replace(" ", "").str.split(",")
filtered_type_col = type_col.apply(lambda l: set(l) - {"call", "song"})
svc_df["pred"] = type_col.apply(lambda l: "call" in l).astype(int)

In [5]:
## svc_df indexed by index
# Build y train-test
# y_df = svc_df.reindex(columns=["id", "pred"]).copy()
# index by index
# y_train, y_test = (
#     y_df[y_df.id.isin(train_ids)].drop(columns=["id"]).squeeze(),
#     y_df[y_df.id.isin(test_ids)].drop(columns=["id"]).squeeze(),
# )
# index by id
# y_train, y_test = (
#     y_df[y_df.id.isin(train_ids)].set_index('id').squeeze(),
#     y_df[y_df.id.isin(test_ids)].set_index('id').squeeze(),
# )

## svc_df indexed by id
# index all (svc_df and y_df) by id
y_df = svc_df["pred"]
y_train, y_test = (
    y_df[y_df.index.isin(train_ids)].squeeze(),
    y_df[y_df.index.isin(test_ids)].squeeze(),
)

### Get a random smaller subset of ids and labels to work with

In [ ]:
# train with a smaller subset of recordings
# index by id
train_ids_ser = pd.Series(train_ids)
train_rand_ids = train_ids_ser[np.random.randint(0,len(train_ids),size=10)].array
y_train_rand = y_train.loc[train_rand_ids]
print(y_train_rand)

# rand_idx = train_ids_ser[np.random.randint(0,len(train_ids),size=10)].index
# y_train.iloc[rand_idx] # check that we have both labels



# test with a smaller subset of recordings
# index by id
test_ids_ser = pd.Series(test_ids)
test_rand_ids = test_ids_ser[np.random.randint(0,len(test_ids),size=10)].array
y_test_rand = y_test.loc[test_rand_ids]
print(y_test_rand)

# test_rand_idx = test_ids_ser[np.random.randint(0,len(test_ids),size=10)].index
# y_test.iloc[test_rand_idx] # check that we have both labels


# all rand ids
rand_ids = np.concatenate((train_rand_ids, test_rand_ids))
rand_ids

## Unpack and filter audio

In [6]:
def highpass_filter(audio, sr):
    # # apply Butter filter
    # butter_coeff_b, butter_coeff_a = signal.butter(3, 1000, btype='highpass', fs=sr) # numerator and denominator
    # butter_audio = signal.lfilter(butter_coeff_b, butter_coeff_a, audio)
    # return butter_audio
    return signal.lfilter(*signal.butter(3, 1000, btype='highpass', fs=sr), audio)

In [7]:
def unpack_audio(id):
    try:
        audio_path = ROOT_PATH / ("data/raw/recordings/" + str(id) + ".mp3")
        # load mp3 as audio timeseries arr
        timeseries,sr = librosa.load(audio_path)
    except FileNotFoundError:
        audio_path = ROOT_PATH / ("data/raw/recordings/" + str(id) + ".wav")
        timeseries,sr = librosa.load(audio_path)

    # timeseries = timeseries[1000:2000] # reduce size
    # high-pass filter on audio timeseries
    timeseries_filt = highpass_filter(timeseries,sr)

    df = pd.DataFrame(timeseries_filt, columns=['val'])
    df.reset_index(inplace=True)
    df['id'] = id
    df = df.reindex(columns=['id','index','val'])
    df.columns = ['id','time','val']
    return df
    # num_samples = len(timeseries_filt)
    # return pd.DataFrame(list(zip([id]*num_samples,range(num_samples),timeseries_filt)),columns=['id','time','val'])

In [25]:
%%timeit
unpack_audio(svc_ids[0])

1.16 s ± 34.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [38]:
%%timeit
unpack_audio(svc_ids[0])

806 ms ± 14.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Save audio timeseries data

In [ ]:
## Trying to unpack all audio in a vertically stacked df (this is WAY too many rows)
# concatenate in batches? concat to what i have already

# audio_df_head = pd.concat([unpack_audio(id) for id in rand_ids], ignore_index=True)
# audio_df_head = pd.concat([unpack_audio(id) for id in svc_ids[rand_idx].array], ignore_index=True)
# audio_df_head

In [8]:
audio_list = [unpack_audio(id) for id in tqdm(svc_ids)]

100%|██████████| 5800/5800 [50:00<00:00,  1.93it/s]


In [ ]:
audio_list

In [10]:
audio_df = pd.concat(audio_list, ignore_index=True) #crashes jupyter

In [60]:
# takes 5h
audio_df = pd.DataFrame(columns = ['id','time','val'])
data_size = len(svc_ids)
buffer_size = 10
df_buffer = [None]*buffer_size
for idx,audio_id in enumerate(tqdm(svc_ids)):
    buffer_idx = idx%buffer_size
    if (buffer_idx == 0):
        audio_df = pd.concat([audio_df]+df_buffer, ignore_index=True)
    df_buffer[buffer_idx] = unpack_audio(audio_id)

# leftover values in buffer
leftover = data_size % buffer_size
if (leftover != 0):
    audio_df = pd.concat([audio_df]+df_buffer[:leftover], ignore_index=True)

 17%|█▋        | 980/5800 [50:17<4:07:18,  3.08s/it]


KeyboardInterrupt: 

In [62]:
# takes 2h
audio_df = pd.DataFrame(columns = ['id','time','val'])
for audio_id in tqdm(svc_ids):  
    audio_df = pd.concat([audio_df,unpack_audio(audio_id)], ignore_index=True)

  1%|▏         | 75/5800 [01:23<1:46:21,  1.11s/it]


KeyboardInterrupt: 

In [9]:
svc_ids[2750]

302534

In [8]:
# 1h20m
# ~80GB
for audio_id in tqdm(svc_ids):  
    id_df = unpack_audio(audio_id)
    id_df.to_json(ROOT_PATH / f"data/processed/audio_timeseries/{audio_id}.json", indent=2, orient='columns')

 47%|████▋     | 2750/5800 [31:22<34:48,  1.46it/s]


OSError: [Errno 28] No space left on device

In [11]:
audio_df.to_json(ROOT_PATH / "data/processed/audio_timeseries.json", indent=2, orient='columns')

In [12]:
## Better (?) way (this takes ~1h):
# dict[id] = df with timeseries data for that id's audio recording

# audio_dict = pd.Series(index=svc_ids.head())
audio_dict = {}
for id in tqdm(svc_ids):
    audio_dict[id] = unpack_audio(id)

100%|██████████| 5800/5800 [1:06:39<00:00,  1.45it/s]


In [17]:
len(audio_dict)

5800

In [20]:
with open(ROOT_PATH / "data/processed/audio_timeseries.pkl", "wb+") as pkl_file:
    pickle.dump(audio_dict, pkl_file, protocol=pickle.HIGHEST_PROTOCOL)

## Extract features using ts-fresh

In [ ]:
# do extract_features in parallel - either using tsfresh n_jobs, or Process Pool Executer
# def parallel_extract_features(id_set, params):
#     # params are a dict of parameters for extract_features
#     with ProcessPoolExecutor(max_workers=10) as ppe:
#         pd.concat(
#             list)
#                 tqdm(
#                     ppe.map(lambda p: extract_features(*p), download_set), total=len(id_set)
#                 )
#             )
#         )

In [ ]:
# manually select features to calculate
# features can be found here: https://tsfresh.readthedocs.io/en/latest/api/tsfresh.feature_extraction.html#tsfresh.feature_extraction.feature_calculators.fft_aggregated

extraction_settings = {
    "abs_energy": None,
    "fft_aggregated": [{"aggtype":"centroid"}, {"aggtype":"kurtosis"}],
    "root_mean_square": None,
    "spkt_welch_density": [{"coeff":2},{"coeff":5},{"coeff":8}]
}

X_df_manual = extract_features(audio_df, column_id='id', column_sort='time',
# X_df_manual = extract_features(audio_df_head, column_id='id', column_sort='time',
                     default_fc_parameters=extraction_settings,
                     # we impute = remove all NaN features automatically
                     impute_function=impute,
                     # turn off parallelization
                     n_jobs=0)
X_df_manual

In [13]:
len(svc_ids)

5800

### Select features

In [ ]:
# get a random sample of 10% of ids from training data for feature selection
percent = .10
train_ids_ser = pd.Series(train_ids)
feat_select_ids = train_ids_ser[np.random.randint(0,len(train_ids),size=int(len(train_ids)*percent))].array

In [ ]:
# try lots of features

# get X and y 
y_for_selected = y_train.loc[feat_select_ids]

# EfficientFCParameters()

X_extracted = extract_features(audio_df[feat_select_ids], column_id='id', column_sort='time', 
                    default_fc_parameters=ComprehensiveFCParameters(),
                    # we impute = remove all NaN features automatically
                    impute_function=impute)
                    # turn off parallelization
                    # n_jobs=0)

In [ ]:
# have ts-fresh figure out which features are good
# presets can be found here: https://tsfresh.readthedocs.io/en/latest/api/tsfresh.feature_extraction.html#tsfresh.feature_extraction.settings.ComprehensiveFCParameters
# with more details here: https://tsfresh.readthedocs.io/en/latest/_modules/tsfresh/feature_extraction/settings.html#MinimalFCParameters

# X_df_autofilt = extract_relevant_features(audio_df_head, y_train.loc[rand_ids], 
# # X_df_autofilt = extract_relevant_features(audio_df_head, y_train.iloc[rand_idx], 
# # X_df_autofilt = extract_relevant_features(audio_df, y_train, 
#                                             column_id='id', column_sort='time', 
#                                             default_fc_parameters=MinimalFCParameters(), n_jobs=0)
X_selected = select_features(X_extracted, y)

# get a dictionary of the good features parameters, to use again later
kind_to_fc_parameters = from_columns(X_selected)

In [ ]:
# extract the selected features from full dataset
X_df = extract_features(audio_df, column_id='id', column_sort='time',
                     default_fc_parameters=kind_to_fc_parameters,
                     # we impute = remove all NaN features automatically
                     impute_function=impute)

In [ ]:
# feature_mapper = DataFrameMapper(
#     [
#         ("id", None),
#         (["gen"], OneHotEncoder()),
#         (["sp"], OneHotEncoder()),
#         (["ssp"], OneHotEncoder()),
#         (["en"], OneHotEncoder()),
#         (["lat"], [MinMaxScaler(), SimpleImputer()]),
#         (["lng"], [MinMaxScaler(), SimpleImputer()]),
#         (["gender"], OneHotEncoder()),
#         (["age"], OneHotEncoder()),
#     ],
#     df_out=True,
# )

In [ ]:
# X_feat_df = feature_mapper.fit_transform(X_df)
# X_train, X_test = (
#     X_df[X_df.id.isin(train_ids)].drop(columns=["id"]),
#     X_df[X_df.id.isin(test_ids)].drop(columns=["id"]),
# )

X_train, X_test = (
    X_df[X_df.index.isin(train_rand_ids)].squeeze(),
    X_df[X_df.index.isin(test_rand_ids)].squeeze(),
)

In [ ]:
lr = LogisticRegression()
lr.fit(X_train, y_train)
# lr.fit(X_train, y_train_rand)

In [ ]:
print(lr.score(X_test, y_test))
# print(lr.score(X_test, y_test_rand))